In [1]:
import numpy as np
import pandas as pd
from src.utils import * 

In [2]:
gbsg = pd.read_csv("../GBSG2")

# On a un jeu de données complet: 
np.where(gbsg.isna()==True)

(array([], dtype=int64), array([], dtype=int64))

In [3]:
data_missing = gbsg.mask(np.random.random(size=gbsg.shape)>0.5)

n_missing = data_missing.isna().sum().sum()
n_missing/(data_missing.shape[0]*data_missing.shape[1])

0.49456665783196396

On détermine $I$ et $J$: 

In [4]:
I, J = gbsg.shape

Trouver $K_1$, $K_2$

In [5]:
# Variables continues
idx_k1 = gbsg.columns[(gbsg.isin([0, 1])==False).any()]

#Variables encodées en dummy variables 
idx_j = gbsg.columns[(gbsg.isin([0, 1])).all()]

In [86]:
class IterativeFAMDImputer(FAMD):
    def __init__(self, data, k1, k2, n_components=2):
        super().__init__(data, k1, k2)
        """Initialisation
        Args:
            n_components (int, optional): _description_. Defaults to 2.
            data (_type_, optional): _description_. Defaults to None.
        """
        self.n_components = n_components
        
    def inital_impute(self, data):
        #Pour les variables continues
        self.sj = self.df_C0.std(axis=0).to_numpy()
        Ximp_C0 = self.df_C0.copy()
        for c in self.df_C0.columns.to_numpy(): 
            Ximp_C0[c] = Ximp_C0[c].fillna(self.df_C0[c].mean())  
        print(Ximp_C0)
        self.df_C0 = Ximp_C0

        # Pour les variables catégorielles 
        self.sqrt_pj = np.sqrt(self.df_categ.sum(axis=0)/self.df_categ.shape[0]).to_numpy()
        Ximp_categ = self.df_categ.copy()  
        for c in self.df_categ.columns.to_numpy(): 
            Ximp_categ[c] =Ximp_categ[c].fillna(self.df_categ[c].sum(axis=0)/data.shape[0])  
        print(Ximp_categ)
        self.df_categ = Ximp_categ / Ximp_categ.sum()
        self.data_concat()
        pass 
        
    def impute(self, n_it, tol=1e-4,verbose=False):
        # Initialisation 
        self.data_concat()
        idx_NA = 1- self.df.isna().astype(int) # 1 si obs 0 sinon
        #Initial imputation 
        self.inital_impute(self.df)
        self.data_concat() # on construit les données avec df_C0 et df_categ remplis 
        self.DM()

        diff = np.inf
        last_chap = np.inf*np.ones_like(idx_NA)
        i= 0
        while i < n_it and diff > tol: 
            Z_p = self.run_famd() #Updating D, M already inside
            X_chap = (Z_p + self.M)@np.sqrt(self.D)
            df = idx_NA*self.df + (1- idx_NA)*X_chap
            self.df= df

            diff = ((X_chap - last_chap)**2).sum().sum()
            last_chap = X_chap
            i +=1 
        if i < n_it: 
            print('Converged in', i)
        else: 
              print('Maximum iterations reached')    
        pass 

In [87]:
famd_algo = FAMD(data=gbsg, k1=idx_k1, k2=idx_j)
famd_algo.run_famd()

,pid,age,size,grade,nodes,pgr,er,rfstime,meno,hormon,status
0,-0.002485,-0.009541,-0.083477,-0.259953,-0.278959,0.004512,0.003273,0.007382,-0.000034,-0.000016,-0.000355
1,0.006887,-0.001826,0.232226,1.981576,0.743902,-0.024163,-0.018125,-0.024987,0.000022,-0.000122,0.001144
2,-0.000374,-0.046791,-0.011145,1.984401,-0.088813,-0.018028,-0.014038,-0.006155,-0.000119,-0.000271,0.000203
3,-0.002406,-0.059489,-0.079264,1.990184,-0.322017,-0.016357,-0.012930,-0.000902,-0.000158,-0.000313,-0.000060
4,0.000121,0.006787,0.003877,-0.269323,0.020144,0.002387,0.001865,0.000652,0.000017,0.000038,-0.000018
...,...,...,...,...,...,...,...,...,...,...,...
681,-0.001529,-0.053892,-0.049864,1.982507,-0.221236,-0.017030,-0.013371,-0.003151,-0.000141,-0.000294,0.000053
682,-0.001978,-0.006012,-0.066485,-0.277420,-0.220380,0.004243,0.003111,0.006128,-0.000023,-0.000004,-0.000292
683,-0.001858,-0.055817,-0.060891,1.977661,-0.258837,-0.016707,-0.013150,-0.002280,-0.000147,-0.000300,0.000010
684,-0.001815,-0.005360,-0.061008,-0.261535,-0.202046,0.003958,0.002905,0.005648,-0.000020,-0.000002,-0.000268


In [88]:
ifamd = IterativeFAMDImputer(n_components=2, data=data_missing, k1=idx_k1, k2=idx_j)

In [89]:
ifamd.impute(100)

             pid        age       size     grade  nodes          pgr  \
0     132.000000  52.996997  18.000000  2.145299    4.8   111.008451   
1    1575.000000  52.996997  20.000000  3.000000   16.0   111.008451   
2    1140.000000  56.000000  29.815126  3.000000    4.8     0.000000   
3     769.000000  45.000000  29.815126  3.000000    1.0     0.000000   
4     999.274448  65.000000  30.000000  2.000000    5.0   111.008451   
..           ...        ...        ...       ...    ...          ...   
681   999.274448  51.000000  30.000000  3.000000    4.8  1152.000000   
682   999.274448  52.996997  26.000000  2.145299    4.8  1356.000000   
683   999.274448  52.996997  35.000000  3.000000    1.0   111.008451   
684   736.000000  52.996997  21.000000  2.145299    4.8   111.008451   
685   999.274448  52.996997  29.815126  2.145299    7.0   111.008451   

              er      rfstime  
0      97.593838  1838.000000  
1      97.593838  1128.878049  
2      97.593838  1603.000000  
3      